<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dylan/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XÂY DỰNG MÔ HÌNH

## Import các thư viện cần thiết

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import string
import lightgbm as lgb
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

Mounted at /gdrive


## Đọc file dữ liệu

In [3]:
train = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/train.csv')
train

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3
...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2
17303,ffddf1f,Technology has changed a lot of ways that we l...,4
17304,fff016d,If you don't like sitting around all day than ...,2
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1


In [4]:
test = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/test.csv')
test_essay_id = test['essay_id']
test

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


## Tiền xử lý dữ liệu

### Làm sạch văn bản

Cần làm sạch văn bản, nhằm chuẩn hóa và loại bỏ những thành phần không cần thiết trước khi tiến hành các bước xử lý tiếp theo.
- Văn bản được chuyển đổi toàn bộ về chữ thường để đảm bảo tính nhất quán và tránh phân biệt giữa chữ hoa và chữ thường.
- Các thẻ HTML, thẻ tên người dùng (bắt đầu bằng `@`), hashtag (bắt đầu bằng `#`), và đường dẫn URL đều được loại bỏ để giữ lại nội dung văn bản thực sự. - Các ký tự đặc biệt, dấu câu và các số trong văn bản, thường không mang lại giá trị ngữ nghĩa, cũng được loại bỏ.
- Khoảng trắng dư thừa hay dấu câu liên tiếp được xử lý và thay thế bằng một ký tự duy nhất.
- Các từ viết tắt được mở rộng thành dạng đầy đủ để đảm bảo tính nhất quán.

In [7]:
def clean_text(text):
    # Chuyển chữ viết hoa thành chữ thường
    text = text.lower()

    # Xóa các thẻ HTML
    text = re.compile(r'<.*?>').sub(r'', text)

    # Xóa các tag tên (mention)
    text = re.sub(r'@\w+\s*', '', text)

    # Xóa hashtag (dấu #)
    text = re.sub(r'#\w+', '', text)

    # Xóa các liên kết URL
    text = re.sub(r'http\S+|www\S+', '', text)

    # Xóa các ký tự không mong muốn như \xa0
    text = text.replace(u'\xa0', ' ')

    # Xóa dấu câu
    text = re.sub(r'[^\w\s]', '', text)

    # Xóa chữ số
    text = re.sub(r'\d+', '', text)

    # Thay thế các khoảng trắng liên tiếp bằng một khoảng trắng duy nhất
    text = re.sub(r'\s+', ' ', text)

    # Mở rộng các từ viết tắt
    # text = expandContractions(text)

    # Thay thế các dấu chấm và dấu phẩy liên tiếp bằng một dấu duy nhất
    text = re.sub(r'\.+', '.', text)
    text = re.sub(r'\,+', ',', text)

    # Xóa các khoảng trắng ở đầu và cuối chuỗi
    text = text.strip()

    return text

train['full_text'] = train['full_text'].apply(clean_text)
test['full_text'] = test['full_text'].apply(clean_text)

### Rút trích đặc trưng đoạn văn

Phân tích về đặc trưng đoạn văn giúp hiểu rõ hơn về cấu trúc văn bản, độ phức tạp và ngữ cảnh văn bản.

In [12]:
import polars as pl

def preprocess_paragraphs(df):
    df = df.explode('paragraph')
    df = df.filter(pl.col('paragraph').str.strip() != "")  # Loại bỏ các đoạn văn trống
    df = df.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_length"))
    df = df.with_columns(
        pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("sentence_count"),
        pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("word_count")
    )
    return df

# Hàm tính toán các đặc trưng từ đoạn văn
def extract_paragraph_features(df):
    features = ['paragraph_length', 'sentence_count', 'word_count']
    aggs = [
        *[pl.col('paragraph_length').filter(pl.col('paragraph_length') >= i).count().alias(f"length_ge_{i}_count") for i in [50, 75, 100, 125, 150, 175, 200, 250, 300, 350, 400, 500, 600, 700]],
        *[pl.col('paragraph_length').filter(pl.col('paragraph_length') <= i).count().alias(f"length_le_{i}_count") for i in [25, 49]],
        *[pl.col(feat).max().alias(f"{feat}_max") for feat in features],
        *[pl.col(feat).mean().alias(f"{feat}_mean") for feat in features],
        *[pl.col(feat).min().alias(f"{feat}_min") for feat in features],
        *[pl.col(feat).first().alias(f"{feat}_first") for feat in features],
        *[pl.col(feat).last().alias(f"{feat}_last") for feat in features],
        *[pl.col(feat).sum().alias(f"{feat}_sum") for feat in features],
        *[pl.col(feat).kurtosis().alias(f"{feat}_kurtosis") for feat in features],
        *[pl.col(feat).quantile(0.25).alias(f"{feat}_q1") for feat in features],
        *[pl.col(feat).quantile(0.75).alias(f"{feat}_q3") for feat in features],
    ]
    df = df.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    return df.to_pandas()

# Chuyển đổi dữ liệu và xử lý các đoạn văn
def process_data(data):
    # Tách các đoạn văn bằng dấu "\n\n" và loại bỏ các đoạn trống
    columns = [pl.col("full_text").str.split(by="\n\n").alias("paragraph")]
    data = pl.from_pandas(data).with_columns(columns)
    return preprocess_paragraphs(data)

# Sử dụng các hàm đã cải tiến trên dữ liệu train và test
train_data = process_data(train)
test_data = process_data(test)

train_feats = extract_paragraph_features(train_data)
test_feats = extract_paragraph_features(test_data)

# Lấy tên các đặc trưng
feature_names = list(filter(lambda x: x not in ['essay_id', 'score'], train_feats.columns))
print('Features Number: ', len(feature_names))
train_feats.head(5)

Features Number:  43


,essay_id,length_ge_50_count,length_ge_75_count,length_ge_100_count,length_ge_125_count,length_ge_150_count,length_ge_175_count,length_ge_200_count,length_ge_250_count,length_ge_300_count,...,word_count_sum,paragraph_length_kurtosis,sentence_count_kurtosis,word_count_kurtosis,paragraph_length_q1,sentence_count_q1,word_count_q1,paragraph_length_q3,sentence_count_q3,word_count_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,489,NaN,NaN,NaN,2597.0,1.0,489.0,2597.0,1.0,489.0
1,000fe60,1,1,1,1,1,1,1,1,1,...,332,NaN,NaN,NaN,1624.0,1.0,332.0,1624.0,1.0,332.0
2,001ab80,1,1,1,1,1,1,1,1,1,...,550,NaN,NaN,NaN,3009.0,1.0,550.0,3009.0,1.0,550.0
3,001bdc0,1,1,1,1,1,1,1,1,1,...,441,NaN,NaN,NaN,2608.0,1.0,441.0,2608.0,1.0,441.0
4,002ba53,1,1,1,1,1,1,1,1,1,...,372,NaN,NaN,NaN,2137.0,1.0,372.0,2137.0,1.0,372.0


## Feature engineering

### Thuật toán TF-IDF (Term Frequency, Inverse Document Frequency)

"TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc \"các thuật ngữ\") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:"
- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng. Cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất. Cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:
$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:
- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

**Reference:** https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77"

In [ ]:
vectorizer = TfidfVectorizer(min_df=.05)
train_tfid = vectorizer.fit_transform(train['full_text'])
test_tfid = vectorizer.transform(test['full_text'])